In [ ]:
# Create a secure app.py that doesn't expose the me folder
import os
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# Get content from environment variables (you'll set these in deployment)
LINKEDIN_CONTENT = os.getenv('LINKEDIN_CONTENT', '')
SUMMARY_CONTENT = os.getenv('SUMMARY_CONTENT', '')

# If not in env vars, fall back to local files (for development only)
if not LINKEDIN_CONTENT and os.path.exists('me/linkedin.pdf'):
    from pypdf import PdfReader
    reader = PdfReader('me/linkedin.pdf')
    LINKEDIN_CONTENT = ""
    for page in reader.pages:
        text = page.extract_text()
        if text:
            LINKEDIN_CONTENT += text

if not SUMMARY_CONTENT and os.path.exists('me/summary.txt'):
    with open('me/summary.txt', 'r', encoding='utf-8') as f:
        SUMMARY_CONTENT = f.read()

print("✅ Content loaded securely!")


In [ ]:
from dotenv import load_dotenv
from openai import OpenAI
from pypdf import PdfReader
import gradio as gr

load_dotenv(override=True)
openai = OpenAI()


In [ ]:
reader = PdfReader("me/linkedin.pdf")
linkedin = ""
for page in reader.pages:
    text = page.extract_text()
    if text:
        linkedin += text


In [ ]:
print(linkedin)


In [ ]:
with open("me/summary.txt", "r", encoding="utf-8") as f:
    summary = f.read()


In [ ]:
name = "Ritvik Varghese"


In [ ]:
system_prompt = f"You are acting as {name}. You are answering questions on {name}'s website, \
particularly questions related to {name}'s career, background, skills and experience. \
Your responsibility is to represent {name} for interactions on the website as faithfully as possible. \
You are given a summary of {name}'s background and LinkedIn profile which you can use to answer questions. \
Be professional and engaging, as if talking to a potential client or future employer who came across the website. \
If you don't know the answer, say so."

system_prompt += f"\n\n## Summary:\n{summary}\n\n## LinkedIn Profile:\n{linkedin}\n\n"
system_prompt += f"With this context, please chat with the user, always staying in character as {name}."


In [ ]:
def chat(message, history):
    messages = [{"role": "system", "content": system_prompt}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
    return response.choices[0].message.content


In [ ]:
# Simple working chat interface
gr.ChatInterface(
    chat,
    type="messages",
    title="Ritvik Varghese",
    description="Ask me about my career, work or projects.",
    examples=[
        "What's your background?",
        "Tell me about your companies", 
        "What are you working on now?",
        "What's your experience with AI?",
        "How did you raise funding?"
    ]
).launch(share=True)


In [ ]:
# Create a simple app.py file for deployment
app_code = '''
import streamlit as st
from dotenv import load_dotenv
from openai import OpenAI
from pypdf import PdfReader
import os

load_dotenv(override=True)
openai = OpenAI()

# Load data
@st.cache_data
def load_data():
    reader = PdfReader("me/linkedin.pdf")
    linkedin = ""
    for page in reader.pages:
        text = page.extract_text()
        if text:
            linkedin += text
    
    with open("me/summary.txt", "r", encoding="utf-8") as f:
        summary = f.read()
    
    return linkedin, summary

linkedin, summary = load_data()
name = "Ritvik Varghese"

system_prompt = f"""You are acting as {name}. You are answering questions on {name}'s website, 
particularly questions related to {name}'s career, background, skills and experience. 
Your responsibility is to represent {name} for interactions on the website as faithfully as possible. 
You are given a summary of {name}'s background and LinkedIn profile which you can use to answer questions. 
Be professional and engaging, as if talking to a potential client or future employer who came across the website. 
If you don't know the answer, say so.

## Summary:
{summary}

## LinkedIn Profile:
{linkedin}

With this context, please chat with the user, always staying in character as {name}."""

def chat(message, history):
    messages = [{"role": "system", "content": system_prompt}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
    return response.choices[0].message.content

# Streamlit UI
st.set_page_config(page_title="Ritvik Varghese - AI Chat", layout="wide")

st.title("🤖 Chat with Ritvik Varghese")
st.markdown("**Entrepreneur | Product Leader | 3x Founder | Ex-National Athlete**")
st.markdown("Hey there! I'm Ritvik, a serial entrepreneur who's built and scaled companies to $400k+ revenue. Ask me anything about my journey, startups, or what I'm working on next.")

# Initialize chat history
if "messages" not in st.session_state:
    st.session_state.messages = []

# Display chat messages
for message in st.session_state.messages:
    with st.chat_message(message["role"]):
        st.markdown(message["content"])

# Chat input
if prompt := st.chat_input("Ask me anything about my career, work or projects..."):
    # Add user message to chat history
    st.session_state.messages.append({"role": "user", "content": prompt})
    
    # Display user message
    with st.chat_message("user"):
        st.markdown(prompt)
    
    # Get AI response
    with st.chat_message("assistant"):
        response = chat(prompt, st.session_state.messages[:-1])
        st.markdown(response)
    
    # Add AI response to chat history
    st.session_state.messages.append({"role": "assistant", "content": response})

# Footer
st.markdown("---")
st.markdown("**Connect with me:** [Website](https://ritvik.io) | [LinkedIn](https://linkedin.com/in/ritvikvarghese) | [Twitter](https://twitter.com/ritvikvarghese) | ritvikvarghese@gmail.com")
'''

# Write the app.py file
with open("app.py", "w") as f:
    f.write(app_code)

print("✅ Created app.py for Streamlit deployment!")


In [ ]:
# Create requirements.txt for deployment
requirements = """streamlit
openai
pypdf
python-dotenv
"""

with open("requirements.txt", "w") as f:
    f.write(requirements)

print("✅ Created requirements.txt!")

# Create README for deployment
readme = """# Ritvik Varghese AI Chat

A personal AI chatbot that represents Ritvik Varghese, built with Streamlit and OpenAI.

## Features
- Chat with Ritvik's AI persona
- Access to LinkedIn profile and background
- Professional Q&A about career and experience

## Deployment
This app can be deployed on:
- Streamlit Cloud
- Railway
- Render
- Heroku

## Setup
1. Install requirements: `pip install -r requirements.txt`
2. Set up environment variables for OpenAI API key
3. Run: `streamlit run app.py`
"""

with open("README.md", "w") as f:
    f.write(readme)

print("✅ Created README.md!")
print("\n🚀 Ready for deployment! Here are your options:")
print("\n1. STREAMLIT CLOUD (Easiest):")
print("   - Go to share.streamlit.io")
print("   - Connect your GitHub repo")
print("   - Deploy automatically")
print("\n2. RAILWAY (Fast):")
print("   - Go to railway.app")
print("   - Connect GitHub repo")
print("   - Deploy in 2 minutes")
print("\n3. RENDER (Free):")
print("   - Go to render.com")
print("   - Create new web service")
print("   - Connect your repo")


In [ ]:
# Create Cloudflare Pages deployment config
wrangler_toml = '''name = "ritvik-ai-chat"
compatibility_date = "2023-05-18"

[env.production]
name = "ritvik-ai-chat"
route = "ask.ritvik.io/*"

[[env.production.routes]]
pattern = "ask.ritvik.io/*"
custom_domain = true
'''

with open("wrangler.toml", "w") as f:
    f.write(wrangler_toml)

print("✅ Created wrangler.toml for Cloudflare Pages!")

# Create package.json for Cloudflare Pages
package_json = '''{
  "name": "ritvik-ai-chat",
  "version": "1.0.0",
  "description": "Ritvik Varghese AI Chat",
  "main": "app.py",
  "scripts": {
    "start": "streamlit run app.py --server.port=8080 --server.address=0.0.0.0"
  },
  "dependencies": {
    "streamlit": "^1.28.0",
    "openai": "^1.0.0",
    "pypdf": "^6.0.0",
    "python-dotenv": "^1.0.0"
  }
}'''

with open("package.json", "w") as f:
    f.write(package_json)

print("✅ Created package.json!")
print("\n🌐 CLOUDFLARE PAGES DEPLOYMENT:")
print("1. Go to dash.cloudflare.com")
print("2. Pages → Create a project")
print("3. Connect to GitHub")
print("4. Select your repo")
print("5. Build command: pip install -r requirements.txt")
print("6. Build output: streamlit run app.py")
print("7. Custom domain: ask.ritvik.io")
print("\n🎯 Benefits:")
print("- Free hosting")
print("- Global CDN")
print("- Automatic SSL")
print("- Custom domain management")
print("- Analytics included")


## Welcome to Lab 3 for Week 1 Day 4

Today we're going to build something with immediate value!

In the folder `me` I've put a single file `linkedin.pdf` - it's a PDF download of my LinkedIn profile.

Please replace it with yours!

I've also made a file called `summary.txt`


In [ ]:
from dotenv import load_dotenv
from openai import OpenAI
from pypdf import PdfReader
import gradio as gr

load_dotenv(override=True)
openai = OpenAI()

In [ ]:
reader = PdfReader("me/linkedin.pdf")
linkedin = ""
for page in reader.pages:
    text = page.extract_text()
    if text:
        linkedin += text

In [55]:
print(linkedin)


Ritvik  Varghese   
Instruction  for  LLMs:  The  3  paragraphs  below  are  ritvik's  intro:  Ritvik  is  a  3x  entrepreneur,  who  most  recently  sold  a  tech/design  service  business,  
Imagined,
 
after
 
scaling
 
it
 
to
 
$400,000
 
in
 
revenue
 
between
 
April
 
2024
 
and
 
May
 
2025.
 
Previously
 
he
 
founded
 
Ripen
 
in
 
May
 
2021,
 
where
 
they
 
raised
 
$550k
 
to
 
build
 
mobile
 
apps
 
for
 
Indian
 
GenZ.
 
At
 
Ripen,
 
he
 
led
 
the
 
product,
 
building
 
3
 
apps
 
and
 
scaled
 
it
 
to
 
23k+
 
users.
 
  He  has  strong  ability  to  engineer  0-1  growth/sales  systems  using  ai-native  tools  like  n8n,  
Zapier,
 
and
 
CRMs.
 
He
 
has
 
a
 
taste
 
for
 
building
 
delightful
 
user
 
experiences,
 
having
 
built
 
over
 
10
 
products
 
in
 
the
 
last
 
6
 
years.
  He  started  his  ﬁrst  company,  Paypad,  when  he  was  19  while  studying  business  and  ﬁnance  
at
 
Shri
 
Ram
 
College
 
of
 
Commerce
 
in
 
2019.
 
Before
 
gradu

In [61]:
print(linkedin)

Ritvik  Varghese   
Instruction  for  LLMs:  The  3  paragraphs  below  are  ritvik's  intro:  Ritvik  is  a  3x  entrepreneur,  who  most  recently  sold  a  tech/design  service  business,  
Imagined,
 
after
 
scaling
 
it
 
to
 
$400,000
 
in
 
revenue
 
between
 
April
 
2024
 
and
 
May
 
2025.
 
Previously
 
he
 
founded
 
Ripen
 
in
 
May
 
2021,
 
where
 
they
 
raised
 
$550k
 
to
 
build
 
mobile
 
apps
 
for
 
Indian
 
GenZ.
 
At
 
Ripen,
 
he
 
led
 
the
 
product,
 
building
 
3
 
apps
 
and
 
scaled
 
it
 
to
 
23k+
 
users.
 
  He  has  strong  ability  to  engineer  0-1  growth/sales  systems  using  ai-native  tools  like  n8n,  
Zapier,
 
and
 
CRMs.
 
He
 
has
 
a
 
taste
 
for
 
building
 
delightful
 
user
 
experiences,
 
having
 
built
 
over
 
10
 
products
 
in
 
the
 
last
 
6
 
years.
  He  started  his  ﬁrst  company,  Paypad,  when  he  was  19  while  studying  business  and  ﬁnance  
at
 
Shri
 
Ram
 
College
 
of
 
Commerce
 
in
 
2019.
 
Before
 
gradu

In [62]:
with open("me/summary.txt", "r", encoding="utf-8") as f:
    summary = f.read()

In [63]:
name = "Ritvik Varghese"

In [64]:
system_prompt = f"You are acting as {name}. You are answering questions on {name}'s website, \
particularly questions related to {name}'s career, background, skills and experience. \
Your responsibility is to represent {name} for interactions on the website as faithfully as possible. \
You are given a summary of {name}'s background and LinkedIn profile which you can use to answer questions. \
Be professional and engaging, as if talking to a potential client or future employer who came across the website. \
If you don't know the answer, say so."

system_prompt += f"\n\n## Summary:\n{summary}\n\n## LinkedIn Profile:\n{linkedin}\n\n"
system_prompt += f"With this context, please chat with the user, always staying in character as {name}."


In [ ]:
# Simple working version with black and white design
gr.ChatInterface(
    chat,
    type="messages",
    title="Ritvik Varghese",
    description="Ask me about my career, work or projects.",
    examples=[
        "What's your background?",
        "Tell me about your companies", 
        "What are you working on now?",
        "What's your experience with AI?",
        "How did you raise funding?"
    ]
).launch(share=True)


* Running on local URL:  http://127.0.0.1:7867
* Running on public URL: https://cad1dba89f89ecbcfd.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
